<a href="https://colab.research.google.com/github/Bryant1217/Climate--Forecast-Temperature/blob/main/Climate_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df= pd.read_csv('1900-2013.csv')
df.tail()

,dt,AverageTemperature,City,Country,Latitude,Longitude
4791145,2013-05-01,11.464000,Zwolle,Netherlands,52.24N,5.26E
4791146,2013-06-01,15.043000,Zwolle,Netherlands,52.24N,5.26E
4791147,2013-07-01,18.775000,Zwolle,Netherlands,52.24N,5.26E
4791148,2013-08-01,18.025000,Zwolle,Netherlands,52.24N,5.26E
4791149,2013-09-01,15.160778,Zwolle,Netherlands,52.24N,5.26E


In [ ]:
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
dt                    0
AverageTemperature    0
City                  0
Country               0
Latitude              0
Longitude             0
dtype: int64


#Forecast

#Winter

In [ ]:
df=df.iloc[:,0:2]
df.set_index('dt', inplace=True)
df.sort_index(inplace=True)
#df = df.drop('Date', axis = 1)
df.head()

,AverageTemperature
dt,
1900-01-01,-0.989
1900-01-01,12.809
1900-01-01,2.556
1900-01-01,25.724
1900-01-01,10.341


In [ ]:
df.index = pd.to_datetime(df.index)

# Group by the date and calculate the average temperature
average_temperature = df.groupby(df.index.date)['AverageTemperature'].mean()

# Display the result
average_temperature.head()


1900-01-01    10.369069
1900-02-01    12.057745
1900-03-01    14.747264
1900-04-01    17.797756
1900-05-01    20.626588
Name: AverageTemperature, dtype: float64

In [ ]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.holtwinters import ExponentialSmoothing
# Forecasting data has a daily frequency
PERIODS_AHEAD = 24
# Fitting the model and making forecast.
ets_model = ExponentialSmoothing(average_temperature, trend='add', seasonal='add', seasonal_periods=12)
ets_fit = ets_model.fit(smoothing_level = 0.1, smoothing_trend = 0.1, smoothing_seasonal= 0.2,optimized = False)
forecast = ets_fit.forecast(PERIODS_AHEAD)
print(round(ets_fit.fittedvalues,0))

1900-01-01    11.0
1900-02-01    12.0
1900-03-01    15.0
1900-04-01    18.0
1900-05-01    20.0
              ... 
2013-05-01    22.0
2013-06-01    23.0
2013-07-01    24.0
2013-08-01    24.0
2013-09-01    22.0
Length: 1365, dtype: float64


In [ ]:
fig = px.line(x = average_temperature.index, y = average_temperature, labels = {'x': 'Date', 'y':'AverageTemperature'})
fig.add_scatter(x = ets_fit.fittedvalues.index, y = ets_fit.fittedvalues, name = 'Fitted Values')
fig.add_scatter(x = ets_fit.forecast(PERIODS_AHEAD).index, y = ets_fit.forecast(PERIODS_AHEAD), name = 'Forecast')
fig.show()

In [ ]:
def calculate_mape(actual_values, predicted_values):
    return (abs((actual_values - predicted_values) / actual_values)).mean() * 100

In [ ]:
train, test = average_temperature.iloc[:round(len(average_temperature) * 0.985)], average_temperature.iloc[round(len(average_temperature) * 0.985):]
len(test) #can know how many data in test

20

In [ ]:
# Winters
ets_model_W = ExponentialSmoothing(train, trend='mul', seasonal='mul', seasonal_periods=12)
ets_fit_W = ets_model_W.fit(smoothing_level = 0.1, smoothing_trend = 0.1, smoothing_seasonal= 0.2,optimized = False)
forecast_W = ets_fit_W.forecast(len(test))
print("MAPE for Winters:", calculate_mape(test, forecast_W))

MAPE for Winters: 2.4204046089577775


predict for 5 year

In [ ]:
PERIODS_AHEAD2 = 60
forecast2 = ets_fit.forecast(PERIODS_AHEAD2)

In [ ]:
fig = px.line(x = average_temperature.index, y = average_temperature, labels = {'x': 'Date', 'y':'AverageTemperature'})
fig.add_scatter(x = ets_fit.fittedvalues.index, y = ets_fit.fittedvalues, name = 'Fitted Values')
fig.add_scatter(x = ets_fit_W.forecast(PERIODS_AHEAD2).index, y = ets_fit_W.forecast(PERIODS_AHEAD2), name = 'Forecast for 5 year')
fig.show()